In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn import preprocessing
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
import scipy.misc
import os
import glob
import file_path

In [2]:
x = []
y = []
x_pixel = []
y_pixel = []
depth = [] #depth
pol = []
time_stamp = []
X_color = []
Y_color = []

#events.tsv: A TSV file where each row corresponds to one event.
# Provided data is: PrimeSense image coordinates (x,y), PrimeSense depth measurement,
# EDVS image coordinates (x,y), Timestamp (in microsenconds), EDVS parity flag

In [3]:
base_directory = r"/home/bru/Dev/DATA/scn2c_take01/take01/"
filename = "events.csv" 
full_path = file_path.file_path(base_directory, filename)

print(full_path)

with open(full_path, encoding="Latin-1") as csvfile:
    reader = list(csv.reader(csvfile, delimiter=","))
print(reader[0])
num_events = len(reader)
num_events_batch = 5000
#print('reader length',num_events)
for i in range(1,num_events):
    X_color.append((reader[i][0]))
    Y_color.append((reader[i][1]))
    depth.append((reader[i][2]))
    x.append(((int(reader[i][3])/128)-0.5)*1000) # convert x_pixels to x in millimeters (according to camera intrinsics of DAVIS128) K = [128, 0, 64; 0, 128, 64; 0, 0, 1]
    y.append(((int(reader[i][4])/128)-0.5)*1000) # convert y_pixels to y in millimeters
    x_pixel.append(int(reader[i][3])) 
    y_pixel.append(int(reader[i][4]))
    time_stamp.append((reader[i][5]))
    pol.append(reader[i][6])
    

img_size = (130, 130)

iEv = 0
iBatch = 0


/home/bru/Dev/DATA/scn2c_take01/take01/events.csv
['369', '388', '1776', '76', '103', '8409', '1']


In [4]:
img = np.zeros(img_size, np.int32) #np.array
count = np.zeros(img_size, np.int32)

depth = np.array(depth)
depth = depth.astype(float)
Max_depth = depth.max()
Min_depth = depth.min()
print(Max_depth,Min_depth)

4463.0 510.0


In [7]:
print(x[1],type(x[1]))
print(type(y))
print(type(depth))

-335.9375 <class 'float'>
<class 'list'>
<class 'numpy.ndarray'>


In [8]:
for i in range(num_events_batch):
    # Need to convert the polarity bit from {0,1} to {-1,+1} and accumulate
    img[int(y[i]),int(x[i])] += (int(depth[i]))
    count[int(y[i]),int(x[i])] += 1

# Compute per-pixel average if count at the pixel is >1
count[count < 1] = 1  # to avoid division by zero
#img = img / count*2
print(img)

IndexError: index -335 is out of bounds for axis 1 with size 130

In [ ]:
fig = plt.figure()
fig.suptitle(num_events_batch)
#maxabsval = np.amax(np.abs(img))
#print(maxabsval)
plt.imshow(img,cmap='gray', clim=(Min_depth,Max_depth))
plt.xlabel("x [pixels]")
plt.ylabel("y [pixels]")
plt.colorbar()
plt.show()

In [ ]:


iEv = 0



In [13]:
from PIL import Image
iEv = 0
iBatch = 0
while True:
    # Check ıf we have processed all events
    if (iEv + num_events_batch > num_events):
        break

    # Select subset of events or data
    iEv += num_events_batch
    iBatch += 1
    depth_batch = depth[iEv+0:iEv+num_events_batch]
    x_batch = x_pixel[iEv+0:iEv+num_events_batch]
    y_batch = y_pixel[iEv+0:iEv+num_events_batch]
    #pol
    #print('x length ', len(x_batch))

    # Reset ımage
    img = np.zeros(img_size, int) #np.array
    count = np.zeros(img_size, int)
    for i in range(num_events_batch):
        # Need to convert the polarity bit from {0,1} to {-1,+1} and accumulate
        img[int(y_batch[i]),int(x_batch[i])] += (int(depth_batch[i]))
        count[int(y_batch[i]),int(x_batch[i])] += 1

    # Compute per-pixel average if count at the pixel is >1
    count[count < 1] = 1  # to avoid division by zero
    img = img / count*2

    # Save ımage
    filename = r"images/depth_event_%08d.png" % iBatch
    img_path = file_path.file_path(base_directory, filename)
    #print(np.shape(img))
    #print(iBatch)
    img8 = (255-(img/Max_depth)*255).astype(np.ubyte)
    img8_r = ((img/Max_depth)*255).astype(np.uint8)
    img = (255-img)

    #plt.imsave(filename, img8_r.astype(np.uint8), cmap='jet_r', origin=(Min_depth,Max_depth))  
    plt.imsave(img_path, img8_r.astype(np.uint8), cmap='jet_r', origin=(Min_depth,Max_depth))   

    #input_path = r"/home/bru/scn3_take07/images/depth_event_%08d.png" % iBatch # input image path
    filename = r"images_nobg/depth_event_nobg_%08d.png" % iBatch
    nobg_img_path = file_path.file_path(base_directory, filename)
    input_array = Image.open(img_path) # load image
    input_array = np.asarray(input_array, dtype=np.uint8)
    img_nobg = np.zeros((130,130,4), dtype=np.uint8)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            [r, g, b, a] = input_array[i, j]
        
            if r==127 and g==0 and b==0 and a==255:
                img_nobg[i,j]=255
            else:
                img_nobg[i,j]=input_array[i,j]
        
            
        image = Image.fromarray(img_nobg, mode='RGBA')
        image.save(nobg_img_path)

        

IndexError: list index out of range

In [1]:
!ffmpeg -framerate 10 -pattern_type glob -i '/home/bru/Dev/DATA/scn2c_take01/take01/images_nobg/*.png' -c:v libxvid -q:v 3 '/home/bru/Dev/DATA/scn2c_take01/take01/images_nobg/000_output.avi'

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li